In [1]:
import numpy as np

In [2]:
from copy import deepcopy

In [48]:
a = np.array([[1,2],[3,4]])

In [68]:
def manhattan(x, y, match=1, mismatch=-1, gap=-1, open_gap=-2):

    weight = np.zeros((3, len(x) + 1, len(y) + 1), dtype=np.float64)
    direction = np.zeros((3, len(x) + 1, len(y) + 1), dtype=int)
    
    weight[0, 1:, 0] = np.arange(1, len(x) + 1) * gap + open_gap # hor
    weight[2, 0, 1:] = np.arange(1, len(y) + 1) * gap + open_gap # ver
    direction[2, 0, 1:] = 2
    weight[[0, 1], 0, 1:], weight[[1, 2], 1:, 0] = -np.inf, -np.inf
    
    for i in range(1, len(x) + 1):
        for j in range(1, len(y) + 1):
#             weight[0, i, j] = max(weight[0, i-1, j] + gap, weight[1, i-1, j] + open_gap + gap,
#                                   weight[2, i-1, j] + open_gap + gap)
            a = weight[0, i-1, j] + gap
            b = weight[1, i-1, j] + open_gap + gap
            c = weight[2, i-1, j] + open_gap + gap
            if a > max(b, c):
                weight[0, i, j] = a
                direction[0, i, j] = 0
            elif b > c:
                weight[0, i, j] = b
                direction[0, i, j] = 1
            else:
                weight[0, i, j] = c
                direction[0, i, j] = 2
#             weight[2, i, j] = max(weight[2, i, j-1] + gap, weight[1, i, j-1] + open_gap + gap,
#                                   weight[0, i, j-1] + open_gap + gap)
            a = weight[2, i, j-1] + gap
            b = weight[1, i, j-1] + open_gap + gap
            c = weight[0, i, j-1] + open_gap + gap
            if a > max(b, c):
                weight[2, i, j] = a
                direction[2, i, j] = 2
            elif b > c:
                weight[2, i, j] = b
                direction[2, i, j] = 1
            else:
                weight[2, i, j] = c
                direction[2, i, j] = 0
#             weight[1, i, j] = max(weight[1, i-1, j-1] + (match if x[i-1] == y[j-1] else mismatch),
#                                   weight[0, i, j], weight[2, i, j])
            a = weight[1, i-1, j-1] + (match if x[i-1] == y[j-1] else mismatch)
            b = weight[0, i, j]
            c = weight[2, i, j]
            if a > max(b, c):
                weight[1, i, j] = a
                direction[1, i, j] = 1
            elif b > c:
                weight[1, i, j] = b
                direction[1, i, j] = 0
            else:
                weight[1, i, j] = c
                direction[1, i, j] = 2
    
    x_res, y_res = '', ''
    lay_num = 1
    i, j = len(x), len(y)
    while i > 0 or j > 0:
        lay_num = direction[lay_num, i, j]
        if lay_num == 1:
            x_res += x[i-1]
            y_res += y[j-1]
            i -= 1
            j -= 1
        elif lay_num == 0:
            x_res += x[i-1]
            y_res += '-'
            i -= 1
        else:
            x_res += '-'
            y_res += y[j-1]
            j -= 1
                
    return weight[1, -1, -1], x_res[::-1], y_res[::-1]

In [69]:
w, s1, s2 = manhattan('aa', 'b', open_gap=1)
print(w)
print(s1)
print(s2)

0.0
a-a
-b-


In [70]:
w, s1, s2 = manhattan('aaaaa', 'bbb', open_gap=1)
print(w)
print(s1)
print(s2)

0.0
aaaa-a-
--b-b-b


In [71]:
w, s1, s2 = manhattan('tccCAGTTATGTCAGgggacacgagcatgcagagac'.upper(),
                             'aattgccgccgtcgttttcagCAGTTATGTCAGatc'.upper(), match=1, gap=-0.5, open_gap=1)
print(w)
print(s1)
print(s2)

36.0
T-C-C-C-A-G-T-T-A-T-G-T-C-A-G-G-G-G-A-C-A-C-G-A-G-C-A-T-G-C-A-G-A-G-A-C-
-A-A-T-T-G-C-C-G-C-C-G-T-C-G-T-T-T-T-C-A-G-C-A-G-T-T-A-T-G-T-C-A-G-A-T-C
